# Check for feasibility

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## Set hyperparameter

In [2]:
EPOCHS = 50

DATASET_PATH = '/Users/shim/dl-python-ImageDetection/dataset/2/'
DATASET_OK_PATTERN = '/Users/shim/dl-python-ImageDetection/dataset/2/OK/'
DATASET_FAIL_PATTERN = '/Users/shim/dl-python-ImageDetection/2/FAIL/'

RESULT_SAVE_PATH = '/Users/shim/dl-python-ImageDetection/dataset/results/'

## Simple Model Setting 

In [11]:
def Model():
    return Sequential([Conv2D(32, (3,3), activation='relu',input_shape=(256, 256, 1)),
                       MaxPool2D(),
                       Conv2D(64, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3,3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

## Import dataset

In [12]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [13]:
def get_file_list(directory, extension):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_list.append(os.path.join(root, file))
    return file_list

In [14]:
ok_list = get_file_list(DATASET_OK_PATTERN, '.png')
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = get_file_list(DATASET_FAIL_PATTERN, '.png')
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Split Train, Validation set

In [15]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_validation = ds.skip(train_size).batch(32)

In [16]:
train_size

662

## Build and Train the Model

In [17]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
model.fit(ds_train, validation_data = ds_validation, epochs=EPOCHS)

Epoch 1/50
21/21 [==============================] - 33s 2s/step - loss: 0.4148 - accuracy: 0.8640 - val_loss: 0.3171 - val_accuracy: 0.8982
Epoch 2/50
21/21 [==============================] - 34s 2s/step - loss: 0.3465 - accuracy: 0.8776 - val_loss: 0.4191 - val_accuracy: 0.8456
Epoch 3/50
21/21 [==============================] - 36s 2s/step - loss: 0.3471 - accuracy: 0.8716 - val_loss: 0.3890 - val_accuracy: 0.8737
Epoch 4/50
21/21 [==============================] - 37s 2s/step - loss: 0.3257 - accuracy: 0.8837 - val_loss: 0.3303 - val_accuracy: 0.8737
Epoch 5/50
21/21 [==============================] - 38s 2s/step - loss: 0.3479 - accuracy: 0.8565 - val_loss: 0.3635 - val_accuracy: 0.8596
Epoch 6/50
21/21 [==============================] - 36s 2s/step - loss: 0.3489 - accuracy: 0.8671 - val_loss: 0.2770 - val_accuracy: 0.8947
Epoch 7/50
21/21 [==============================] - 37s 2s/step - loss: 0.3397 - accuracy: 0.8731 - val_loss: 0.3675 - val_accuracy: 0.8667
Epoch 8/50
21/21 [==

2023-10-19 15:13:03.655179: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 869 of 947
2023-10-19 15:13:03.657471: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 870 of 947
2023-10-19 15:13:03.692445: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


21/21 [==============================] - 377s 3s/step - loss: 0.2508 - accuracy: 0.8958 - val_loss: 0.2925 - val_accuracy: 0.8842
Epoch 39/50
21/21 [==============================] - 56s 3s/step - loss: 0.2570 - accuracy: 0.8973 - val_loss: 0.2347 - val_accuracy: 0.9088
Epoch 40/50


2023-10-19 15:14:17.823584: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 829 of 947
2023-10-19 15:14:17.883866: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2023-10-19 15:14:17.883906: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1 of 1024
2023-10-19 15:14:17.884682: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


21/21 [==============================] - 50s 2s/step - loss: 0.2581 - accuracy: 0.9079 - val_loss: 0.2347 - val_accuracy: 0.9053
Epoch 41/50
21/21 [==============================] - 204s 10s/step - loss: 0.2521 - accuracy: 0.9154 - val_loss: 0.1639 - val_accuracy: 0.9439
Epoch 42/50
21/21 [==============================] - 134s 7s/step - loss: 0.2647 - accuracy: 0.8973 - val_loss: 0.2304 - val_accuracy: 0.9088
Epoch 43/50
21/21 [==============================] - 267s 13s/step - loss: 0.2238 - accuracy: 0.9245 - val_loss: 0.2424 - val_accuracy: 0.8912
Epoch 44/50
21/21 [==============================] - 153s 8s/step - loss: 0.2290 - accuracy: 0.9230 - val_loss: 0.2386 - val_accuracy: 0.9018
Epoch 45/50
21/21 [==============================] - 1242s 62s/step - loss: 0.2444 - accuracy: 0.9094 - val_loss: 0.2232 - val_accuracy: 0.9298
Epoch 46/50
21/21 [==============================] - 1615s 81s/step - loss: 0.2544 - accuracy: 0.9048 - val_loss: 0.2829 - val_accuracy: 0.8877
Epoch 47/50
2

## Save results as images

In [24]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_validation:
    preds = model.predict(imgs)
    for idx in range (imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
        cv2.imwrite(path + '/%.f_%04d.png' % (y, index), imgs[idx].numpy() *255)
        index +=1

1/1 [==============================] - 0s 381ms/step
